In [1]:
import arcgis
import getpass
import pandas as pd
from arcgis.gis import GIS

In [2]:
password = getpass.getpass("Enter your password: ")

Enter your password: ········


In [3]:
gis = GIS("https://www.arcgis.com", "haowanyu", password)

## Import Data From Annual_Parking_Study_Data.csv

In [4]:
APSD = pd.read_csv("../Datasets/Annual_Parking_Study_Data.csv")

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
APSD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145355 entries, 0 to 145354
Data columns (total 22 columns):
Elmntkey                  145277 non-null float64
Study_Area                145329 non-null object
Sub_Area                  132175 non-null object
Date Time                 145355 non-null object
Side                      145316 non-null object
Unitdesc                  145329 non-null object
Peak Hour_SDOT            15222 non-null object
Parking_Spaces            145057 non-null float64
Total_Vehicle_Count       144870 non-null float64
Dp_Count                  116353 non-null float64
Rpz_Count                 33500 non-null float64
TG_Car2Go                 34179 non-null float64
BMW_DN                    13139 non-null float64
Field Notes               15704 non-null object
Construction              145342 non-null object
Event Closure             145342 non-null object
Subarea Label             145355 non-null object
Study Year                145329 non-null float64
Peak

In [6]:
APSD.head()

,Elmntkey,Study_Area,Sub_Area,Date Time,Side,Unitdesc,Peak Hour_SDOT,Parking_Spaces,Total_Vehicle_Count,Dp_Count,...,BMW_DN,Field Notes,Construction,Event Closure,Subarea Label,Study Year,Peak Hour? (Yes or No),RPZ Blocks,CSM,Time Stamp
0,1577.0,12th Ave - Weekday,NaN,4-08-14 14:00:00,W,12TH AVE BETWEEN E COLUMBIA ST AND E MARION ST,NaN,8.0,9.0,7.0,...,NaN,NaN,No,No,12th Ave - Weekday,2014.0,No,NaN,NaN,NaN
1,1577.0,12th Ave - Weekday,NaN,4-08-14 15:00:00,W,12TH AVE BETWEEN E COLUMBIA ST AND E MARION ST,NaN,8.0,8.0,5.0,...,NaN,NaN,No,No,12th Ave - Weekday,2014.0,No,NaN,NaN,NaN
2,1577.0,12th Ave - Weekday,NaN,4-08-14 13:00:00,W,12TH AVE BETWEEN E COLUMBIA ST AND E MARION ST,NaN,8.0,6.0,4.0,...,NaN,NaN,No,No,12th Ave - Weekday,2014.0,No,NaN,NaN,NaN
3,1577.0,12th Ave - Weekday,NaN,4-08-14 09:00:00,W,12TH AVE BETWEEN E COLUMBIA ST AND E MARION ST,NaN,8.0,6.0,3.0,...,NaN,NaN,No,No,12th Ave - Weekday,2014.0,No,NaN,NaN,NaN
4,1577.0,12th Ave - Weekday,NaN,4-08-14 12:00:00,W,12TH AVE BETWEEN E COLUMBIA ST AND E MARION ST,NaN,8.0,6.0,3.0,...,NaN,NaN,No,No,12th Ave - Weekday,2014.0,No,NaN,NaN,NaN


##### Select the 'Elmnkey' Column

In [7]:
Elmntkey = APSD[['Elmntkey']].drop_duplicates()

In [8]:
Elmntkey.head()

,Elmntkey
0,1577.0
6,24553.0
7,24554.0
9,24557.0
13,1578.0


In [9]:
Elmntkey.columns = ['ELMNTKEY']

In [10]:
Elmntkey.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2232 entries, 0 to 145340
Data columns (total 1 columns):
ELMNTKEY    2231 non-null float64
dtypes: float64(1)
memory usage: 34.9 KB


## Import Data From Blockface.csv

In [11]:
Blockface = pd.read_csv("../Datasets/Blockface.csv")

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (34,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
Blockface.head()

,OBJECTID,ELMNTKEY,SEGKEY,UNITID,UNITID2,UNITDESC,SIDE,BLOCK_ID,BLOCK_NBR,CSM,...,SUBAREA,START_TIME_WKD,END_TIME_WKD,START_TIME_SAT,END_TIME_SAT,PRIMARYDISTRICTCD,SECONDARYDISTRICTCD,OVERRIDEYN,OVERRIDECOMMENT,SHAPE_Length
0,1,42410,19829,15755,70,S DEARBORN ST BETWEEN 7TH AVE S AND 8TH AVE S,S,NaN,700,Y,...,Edge,08AM,08PM,08AM,08PM,DISTRICT2,,N,NaN,275.130989
1,2,2794,2773,960,20,2ND AVE N BETWEEN JOHN ST AND THOMAS ST,E,N02-02,200,Y,...,Edge,08AM,08PM,08AM,08PM,DISTRICT7,,N,NaN,424.952259
2,3,85957,17987,13872,120,NE CAMPUS EB PY BETWEEN 12TH AVE NE AND BROOKL...,N,NECP-12,1200,Y,...,Core,08AM,08PM,08AM,08PM,DISTRICT4,,N,NaN,245.027944
3,4,52513,7237,3885,150,5TH AVE BETWEEN PIKE ST AND PINE ST,SW,05-15,1500,Y,...,Retail,08AM,08PM,08AM,08PM,DISTRICT7,,N,NaN,400.491866
4,5,12034,12025,8920,80,OCCIDENTAL AVE S BETWEEN RAILROAD WAY S AND S ...,E,SOO-09,800,Y,...,Edge,08AM,06PM,08AM,06PM,DISTRICT2,,N,NaN,1013.058630


In [13]:
Blockface.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47679 entries, 0 to 47678
Data columns (total 46 columns):
OBJECTID               47679 non-null int64
ELMNTKEY               47679 non-null int64
SEGKEY                 47679 non-null int64
UNITID                 47679 non-null int64
UNITID2                47679 non-null int64
UNITDESC               47679 non-null object
SIDE                   47679 non-null object
BLOCK_ID               2383 non-null object
BLOCK_NBR              47679 non-null int64
CSM                    47679 non-null object
PARKING_CATEGORY       47679 non-null object
LOAD                   47679 non-null int64
ZONE                   47679 non-null int64
TOTAL_ZONES            47679 non-null int64
WKD_RATE1              47679 non-null float64
WKD_START1             47679 non-null int64
WKD_END1               47679 non-null int64
WKD_RATE2              47679 non-null float64
WKD_START2             47679 non-null int64
WKD_END2               47679 non-null int64
WKD

In [14]:
Blockface_slice = Blockface[['ELMNTKEY','SEGKEY']]

In [15]:
Blockface_slice.head()

,ELMNTKEY,SEGKEY
0,42410,19829
1,2794,2773
2,85957,17987
3,52513,7237
4,12034,12025


In [16]:
Blockface.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47679 entries, 0 to 47678
Data columns (total 46 columns):
OBJECTID               47679 non-null int64
ELMNTKEY               47679 non-null int64
SEGKEY                 47679 non-null int64
UNITID                 47679 non-null int64
UNITID2                47679 non-null int64
UNITDESC               47679 non-null object
SIDE                   47679 non-null object
BLOCK_ID               2383 non-null object
BLOCK_NBR              47679 non-null int64
CSM                    47679 non-null object
PARKING_CATEGORY       47679 non-null object
LOAD                   47679 non-null int64
ZONE                   47679 non-null int64
TOTAL_ZONES            47679 non-null int64
WKD_RATE1              47679 non-null float64
WKD_START1             47679 non-null int64
WKD_END1               47679 non-null int64
WKD_RATE2              47679 non-null float64
WKD_START2             47679 non-null int64
WKD_END2               47679 non-null int64
WKD

In [17]:
Blockface_slice['ELMNTKEY'] = Blockface_slice['ELMNTKEY'].astype('float64')

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
Blockface_slice.dtypes

ELMNTKEY    float64
SEGKEY        int64
dtype: object

### Select the rows that the 'ELMNTKEY' value is in Elmnkey

In [19]:
# ba_inner = pd.merge(Elmntkey, Blockface_slice, how='inner', on=['ELMNTKEY'])
ba_inner = Blockface_slice.loc[Blockface_slice['ELMNTKEY'].isin(Elmntkey['ELMNTKEY'])]

In [20]:
ba_inner.head()

,ELMNTKEY,SEGKEY
1,2794.0,2773
4,12034.0,12025
5,80890.0,12905
8,30970.0,8160
9,58078.0,12771


In [21]:
ba_inner.dropna(inplace=True)

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
ba_inner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2223 entries, 1 to 47656
Data columns (total 2 columns):
ELMNTKEY    2223 non-null float64
SEGKEY      2223 non-null int64
dtypes: float64(1), int64(1)
memory usage: 52.1 KB


In [23]:
SEGKEY = ba_inner[['SEGKEY']]
SEGKEY.head()

,SEGKEY
1,2773
4,12025
5,12905
8,8160
9,12771


In [24]:
SEGKEY.columns = ['COMPKEY']

## Compare the csv file and the data from shapefile

In [25]:
Seattle_Streets = pd.read_csv("../Datasets/Seattle_Streets.csv")

In [26]:
Seattle_Streets.head()

,OBJECTID,ARTCLASS,COMPKEY,UNITIDSORT,UNITDESC,STNAME_ORD,XSTRLO,XSTRHI,ARTDESCRIPT,OWNER,...,DIRHI,NATIONHWYSYS,STREETTYPE,PVMTCONDINDX1,PVMTCONDINDX2,TRANCLASS,TRANDESCRIPT,SLOPE_PCT,PVMTCATEGORY,SHAPE_Length
0,1,2.0,1002,100080.0,1ST AVE BETWEEN COLUMBIA ST AND MARION ST,1ST AVE,COLUMBIA ST,MARION ST,Minor Arterial,,...,SE,N,Downtown Neighborhood,19.0,0.0,1,PRINCIPAL TRANSIT ROUTE,1.0,ART,305.996038
1,2,2.0,1020,100280.0,1ST AVE BETWEEN CLAY ST AND BROAD ST,1ST AVE,CLAY ST,BROAD ST,Minor Arterial,,...,SE,N,Downtown Neighborhood,48.0,0.0,2,MAJOR TRANSIT ROUTE,1.0,ART,313.167074
2,3,1.0,1029,150050.0,1ST AVE N BETWEEN REPUBLICAN ST AND MERCER ST,1ST AVE N,REPUBLICAN ST,MERCER ST,Principal Arterial,,...,S,N,Urban Village Main,61.0,0.0,2,MAJOR TRANSIT ROUTE,2.0,ART,476.654551
3,4,0.0,1057,150263.0,1ST AVE N BETWEEN DEAD END 2 AND NEWELL N ST,1ST AVE N,DEAD END 2,NEWELL N ST,Not Designated,,...,S,N,Neighborhood Yield Street,0.0,0.0,0,NOT DESIGNATED,18.0,NON-ART,124.893068
4,5,0.0,1069,200500.0,1ST AVE NE BETWEEN NE 50TH ST AND NE 51ST ST,1ST AVE NE,NE 50TH ST,NE 51ST ST,Not Designated,,...,S,N,Neighborhood Yield Street,35.0,0.0,0,NOT DESIGNATED,1.0,NON-ART,267.019097


## Search the uploaded content and import the spatial dataframe

In [27]:
streets = gis.content.search("Seattle_Street", "feature service")[0]

In [28]:
streets=streets.layers[0]

In [29]:
sdf = pd.DataFrame.spatial.from_layer(streets)

In [30]:
sdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23811 entries, 0 to 23810
Data columns (total 38 columns):
ARTCLASS         23811 non-null int64
ARTDESCRIP       23811 non-null object
BLOCKNBR         23811 non-null int64
COMPKEY          23811 non-null int64
DIRHI            23811 non-null object
DIRLO            23811 non-null object
FID              23811 non-null int64
FLOW             23811 non-null object
INTKEYLO         23811 non-null int64
INTRHI           23811 non-null object
INTRLO           23811 non-null object
NATIONHWYS       23811 non-null object
OBJECTID         23811 non-null int64
ONEWAY           23811 non-null object
ONEWAYDIR        23811 non-null object
OWNER            23811 non-null object
PVMTCATEGO       23811 non-null object
PVMTCONDIN       23811 non-null int64
PVMTCOND_1       23811 non-null int64
SEGDIR           23811 non-null object
SEGLENGTH        23811 non-null float64
SHAPE            23811 non-null geometry
SHAPE_Leng       23811 non-null float6

In [31]:
sdf.dtypes

ARTCLASS            int64
ARTDESCRIP         object
BLOCKNBR            int64
COMPKEY             int64
DIRHI              object
DIRLO              object
FID                 int64
FLOW               object
INTKEYLO            int64
INTRHI             object
INTRLO             object
NATIONHWYS         object
OBJECTID            int64
ONEWAY             object
ONEWAYDIR          object
OWNER              object
PVMTCATEGO         object
PVMTCONDIN          int64
PVMTCOND_1          int64
SEGDIR             object
SEGLENGTH         float64
SHAPE            geometry
SHAPE_Leng        float64
SLOPE_PCT           int64
SPEEDLIMIT          int64
STATUS             object
STNAME_ORD         object
STREETTYPE         object
SURFACETYP         object
SURFACET_1         object
SURFACEWID          int64
Shape__Length     float64
TRANCLASS           int64
TRANDESCRI         object
UNITDESC           object
UNITIDSORT         object
XSTRHI             object
XSTRLO             object
dtype: objec

In [32]:
sdf.spatial.validate()

True

In [34]:
sdf.spatial.geometry_type

['Polyline']

In [35]:
sdf.head().spatial.geometry_type

E:\Anaconda\lib\site-packages\arcgis\features\geo\_accessor.py:858: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._data[col] = self._data[col]


['Polyline']

## Select the rows again

In [36]:
# street_block_merge = pd.merge(sdf,SEGKEY, how='inner', on=['COMPKEY'])
SEGKEY.head()
street_block_merge = sdf.loc[sdf['COMPKEY'].isin(SEGKEY['COMPKEY'])]

In [37]:
street_block_merge.dropna(inplace=True)
# street_block_merge.spatial.validate()

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


False

In [38]:
street_block_merge.spatial.geometry_type

E:\Anaconda\lib\site-packages\arcgis\features\geo\_accessor.py:858: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._data[col] = self._data[col]


['Polyline']

In [43]:
street_block_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1297 entries, 4 to 23739
Data columns (total 38 columns):
ARTCLASS         1297 non-null int64
ARTDESCRIP       1297 non-null object
BLOCKNBR         1297 non-null int64
COMPKEY          1297 non-null int64
DIRHI            1297 non-null object
DIRLO            1297 non-null object
FID              1297 non-null int64
FLOW             1297 non-null object
INTKEYLO         1297 non-null int64
INTRHI           1297 non-null object
INTRLO           1297 non-null object
NATIONHWYS       1297 non-null object
OBJECTID         1297 non-null int64
ONEWAY           1297 non-null object
ONEWAYDIR        1297 non-null object
OWNER            1297 non-null object
PVMTCATEGO       1297 non-null object
PVMTCONDIN       1297 non-null int64
PVMTCOND_1       1297 non-null int64
SEGDIR           1297 non-null object
SEGLENGTH        1297 non-null float64
SHAPE            1297 non-null geometry
SHAPE_Leng       1297 non-null float64
SLOPE_PCT        1297 

## Draw on the map

In [47]:
Seattle = gis.map("Seattle", 10)
Seattle

MapView(layout=Layout(height='400px', width='100%'), zoom=10.0)

In [49]:
street_block_merge.spatial.plot(map_widget = Seattle, symbol_type='simple', colors='autumn')

True